Q1. Model the game of Snakes and Ladders (single-player game) as a Markov Process. Write out it’s state space and structure of transition probabilities.

Answer:

State space = {k} where $1<=k<=100$ inidicating the position of the player.
$k=100$ is terminal state.

Here we define two dictionaries ladder and snake, where (key,value) pair are starts and ends of ladder or snake.

Transition matrix $P(k,k'):N \times S \rightarrow [0,1] $ are defined as

$P(k,k') =  \begin{cases}
   1  & k \in dic.keys(),\ k' = dic[k], dic = snake \ or \ ladder\\
   1/6   & k \notin dic.keys(),k'-k \leq 6\\
   0     & else
\end{cases}$

